In [3]:
!pip install midi-player

In [41]:
from miditok import REMI, Structured, TokenizerConfig
from miditok.utils import get_midi_programs
from symusic import Score
from pathlib import Path

# midi_file = '/home/mm/midi/maestro/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi'
midi_file = 'lmd_matched/B/B/O/TRBBOFI128F4291250/bc8830293f19b2e3941ed0b2f8bd6f00.mid'
score = Score(midi_file) 

config = TokenizerConfig(
    use_programs=True, 
    use_time_signatures=True,
    one_token_stream_for_programs=True
)
tokenizer = REMI(config)

score = tokenizer(tokenizer(score))

score.dump_midi("temp.mid")

In [46]:
len(tokenizer.vocab)

484

In [ ]:
from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced

MIDIPlayer("temp.mid", 400)  
# MIDIPlayer(midi_file, 160, styler=cifka_advanced, title='My Player')

In [ ]:

# Constructs the vocabulary with BPE, from the tokenized files
tokenizer.learn_bpe(
    vocab_size=500,
    tokens_paths=list(Path("/home/mm/midi/maestro_tokens/").glob("**/*.json")),
    start_from_empty_voc=False,
)

# Saving our tokenizer, to retrieve it back later with the load_params method
tokenizer.save_params(Path("../data/tokenizer_params.json"))

In [ ]:
!ls /home/mm/midi/maestro_tokens/